In [1]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
from IPython.display import HTML
import networkx as nx

from numpy.fft import fft, ifft, fftfreq

from scipy.spatial.distance import pdist, squareform
from scipy.stats import rankdata, ttest_rel, ttest_1samp, pearsonr,spearmanr
from scipy.signal import hilbert

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlinesÒ
import matplotlib.transforms as mtransforms
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation

import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn.plotting import plot_glass_brain, plot_stat_map, view_img, view_img_on_surf
from nilearn.image import new_img_like

from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import isc, isfc, isps, fdr, threshold, phase_randomize, circle_shift, _butter_bandpass_filter, _phase_mean_angle, _phase_vector_length

from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings('ignore')


In [3]:

base_dir = '/Users/li/Desktop/debate/braindata'

sub_list = [f'sub-{x:0>3d}' for x in range(13,51)]
sub_list.remove('sub-021')

subs_roi_data = []
for sub in sub_list:
    csv_file = f'/Volumes/Li/task-debate/braindata/denoised 5/parcel data/Schaefer 200 combine 6 runs/{sub}_combined_time-series_Schaefer2018_200Parcels_7Networks.csv'
    sub_data = pd.read_csv(csv_file)
    subs_roi_data.append(sub_data.values)

all_brain_data = np.array(subs_roi_data)

mask_file = '/Users/li/Desktop/template/Schaefer/tpl-MNI152NLin2009cAsym_res-02_atlas-Schaefer2018_desc-200Parcels7Networks_dseg.nii.gz'
mask_img = nib.load(mask_file)
mask_data = mask_img.get_fdata()

nw_labels = pd.read_csv('/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
roi_name = list(nw_labels['ROI Name'])


In [4]:
bahav_data_dir = '/Users/li/Desktop/task-debate/behavdata'

sub_list_num = list(range(13,51))
sub_list_num.remove(21)

# time_points = list(range(0,3000,60)) + [2986]  # every 1 minute
time_points = list(range(0,3000,120)) + [2986] # every 2 min
# time_points = list(range(0,3000,300)) + [2986] # every 5 min
# time_points = [0, 252, 500, 772, 1098, 1484, 1892, 2464, 2986] # every speaker
# time_points = list(range(0,2987,2)) # every TR

# time_points = [0,80,168,208,252, 
#                326,364,464,500,
#                538,588,686,772,
#                860,986,1026,1098,
#                1204,1250,1406,1484,
#                1578,1722,1810,1892,
#                1972,2114,2216,2464,
#                2628,2756,2986] 

all_subject_data = []
for sub in sub_list_num:
    file_path = os.path.join(bahav_data_dir, 'during_scan', 'combined_6runs_per_TR_filter', f'subject_{sub}_TR_rate.csv')
    
    df = pd.read_csv(file_path)
    sub_data = df[df['time'].isin(time_points)]
    all_subject_data.append(list(sub_data['rate']))
    
attitude = pd.DataFrame(all_subject_data)

start_attitude = pd.DataFrame(attitude)[0]
start_attitude_SM = -np.abs(start_attitude.values[:, np.newaxis] - start_attitude.values)

attitude_change = attitude.diff(axis=1)
attitude_change = attitude_change.drop(attitude_change.columns[0], axis=1)
print(attitude_change.shape)

attitude_change_distances = -pdist(attitude_change)
# attitude_change_distances = -np.sqrt(pdist(attitude_change))
# attitude_change_distances = -np.log(pdist(attitude_change))
attitude_change_SM = squareform(attitude_change_distances)
print(attitude_change_SM.shape)

# sns.heatmap(attitude_change_SM)


(37, 25)
(37, 37)


In [6]:

subjects = list(range(13,51))
subjects.remove(21)

personality = pd.read_csv('/Users/li/Desktop/task-debate/behavdata/questionire_data/personality.csv')

selected_data = personality[personality['sub'].isin(subjects)]
selected_data = selected_data.set_index('sub').loc[subjects]

ages = selected_data['age'].values
age_diff_matrix = np.abs(ages[:, np.newaxis] - ages)

sex = selected_data['sex'].values
sex_diff_matrix = np.abs(sex[:, np.newaxis] - sex)

IUS = selected_data['IUS'].values
joint_IUS = (IUS[:, np.newaxis] + IUS)/2



In [10]:
subs_dISFC = np.load('/Users/li/Desktop/debate2025/results/subs_dISFC_seed89.npy')
subs_dISFC = np.arctanh(subs_dISFC)
intersub_dISFC_similarity = []
for edge in range(subs_dISFC.shape[1]):
    intersub_dISFC_similarity .append(Adjacency(1 - pairwise_distances(subs_dISFC[:, edge, :], metric='correlation'), matrix_type='similarity'))
brain_ISC = Adjacency(intersub_dISFC_similarity )

brain_ISC_np = np.array(brain_ISC.squareform())
print(brain_ISC_np.shape)

brain_ISC_Z_np = np.arctanh(brain_ISC_np)


(199, 37, 37)


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np

def regress_out(vec, control_matrices, upper_triangle_indices, standardize):
    """
    对给定的向量进行回归，剥离控制变量的影响，先标准化再回归（可选）。
    
    参数:
    - vec: np.array, 待剥离影响的向量
    - control_matrices: list of np.array, 控制变量的矩阵列表
    - upper_triangle_indices: tuple, 矩阵上三角的索引
    - standardize: bool, 是否先标准化数据，默认值为 True
    
    返回:
    - residuals: np.array, 回归后的残差
    """
    if control_matrices is None or len(control_matrices) == 0:
        return vec  # 如果没有控制变量，直接返回原始向量
    
    # 如果需要标准化，先标准化 vec
    if standardize:
        scaler = StandardScaler()
        vec_scaled = scaler.fit_transform(vec.reshape(-1, 1)).flatten()
    else:
        vec_scaled = vec
    
    # 收集控制变量的标准化版本（如果需要）
    control_vectors = []
    for control_matrix in control_matrices:
        control_vec = control_matrix[upper_triangle_indices]
        if standardize:
            control_vec_scaled = scaler.fit_transform(control_vec.reshape(-1, 1)).flatten()
        else:
            control_vec_scaled = control_vec
        control_vectors.append(control_vec_scaled)
    
    # 拼接控制变量矩阵
    control_matrix = np.column_stack(control_vectors)

    # 执行回归
    model = LinearRegression().fit(control_matrix, vec_scaled)
    
    # 计算残差
    residuals = vec_scaled - model.predict(control_matrix)
    
    return residuals

def mantel_with_multiple_controls(matrix1, matrix2, control_matrices=None, corr_type='spearman', permutations=1000, tail=2, standardize=True):
    """
    执行带有多个控制变量的 Mantel Test，包含置换检验。
    
    参数:
    - matrix1: np.array, 第一个距离矩阵 (NxN)
    - matrix2: np.array, 第二个距离矩阵 (NxN)
    - control_matrices: list of np.array, 控制变量的矩阵列表（默认为 None）
    - permutations: int, 置换次数
    - tail: int, p值的类型，1 表示单尾，2 表示双尾
    
    返回:
    - r_obs: 观察到的 Pearson 或 Spearman 相关系数
    - p_value: 置换检验的 p 值
    """
    # 确保输入矩阵是方阵
    assert matrix1.shape == matrix2.shape, "两个矩阵的形状必须相同"
    assert matrix1.shape[0] == matrix1.shape[1], "输入必须是方阵"
    
    # 提取上三角部分（不包括对角线）
    upper_triangle_indices = np.triu_indices_from(matrix1, k=1)
    vec1 = matrix1[upper_triangle_indices]
    vec2 = matrix2[upper_triangle_indices]
    
    # 对每个控制矩阵进行回归，剥离控制变量的影响
    residuals1 = regress_out(vec1, control_matrices, upper_triangle_indices,standardize=standardize)
    residuals2 = regress_out(vec2, control_matrices, upper_triangle_indices,standardize=standardize)
    
    # 计算观察到的相关系数
    if corr_type == 'pearson':
        r_obs, _ = pearsonr(residuals1, residuals2)
    elif corr_type == 'spearman':
        r_obs, _ = spearmanr(residuals1, residuals2)
    
    # 进行置换检验
    permuted_r = []
    n = matrix1.shape[0]
    
    for _ in range(permutations):
        # 随机打乱行列索引
        perm_indices = np.random.permutation(n)
        
        # 重新排列矩阵
        perm_matrix2 = matrix2[np.ix_(perm_indices, perm_indices)]
        
        # 对每个控制变量矩阵也进行相同的置换（如果存在控制变量）
        perm_control_matrices = []
        if control_matrices is not None:
            for control_matrix in control_matrices:
                perm_control_matrix = control_matrix[np.ix_(perm_indices, perm_indices)]
                perm_control_matrices.append(perm_control_matrix)
        
        # 提取置换后的上三角部分
        perm_vec2 = perm_matrix2[upper_triangle_indices]
        
        # 对置换后的矩阵进行回归，剥离控制变量的影响
        perm_residuals2 = regress_out(perm_vec2, perm_control_matrices, upper_triangle_indices, standardize=standardize)
        
        # 计算置换后的相关系数
        if corr_type == 'pearson':
            r_perm, _ = pearsonr(residuals1, perm_residuals2)
        elif corr_type == 'spearman':
            r_perm, _ = spearmanr(residuals1, perm_residuals2)
        
        permuted_r.append(r_perm)
    
    # 计算双尾或单尾 p 值
    p_value = _calc_pvalue(np.array(permuted_r), r_obs, tail)
    
    return r_obs, p_value

def _calc_pvalue(all_p, stat, tail):
    """计算基于置换分布的 p 值
    
    参数：
    - all_p: 置换分布的相关系数列表
    - stat: 观察到的统计量（如统计结果中的相关系数）
    - tail: (int) 1 或 2，表示单尾或双尾 p 值
    
    返回：
    - p_value: 计算得到的 p 值
    """
    denom = float(len(all_p)) + 1
    if tail == 1:
        numer = np.sum(all_p >= stat) + 1 if stat >= 0 else np.sum(all_p <= stat) + 1
    elif tail == 2:
        numer = np.sum(np.abs(all_p) >= np.abs(stat)) + 1
    else:
        raise ValueError("tail 必须是 1 或 2")
    return numer / denom


In [20]:

rlist1, plist1 = [],[]
for roi in range(199):

    r, p = mantel_with_multiple_controls(attitude_change_SM, brain_ISC_Z_np[roi],
                                        standardize=True, corr_type='pearson',
                                        control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                        permutations=10000,tail=2)
    if p <0.05:
        print(f'{roi},{r:.3f},{p}')

    rlist1.append(r)
    plist1.append(p)

5,0.177,0.015498450154984501
13,0.155,0.042495750424957505
22,0.170,0.0380961903809619
31,0.155,0.024897510248975102
34,0.174,0.022397760223977603
37,0.163,0.0461953804619538
51,0.142,0.0377962203779622
57,0.218,0.0029997000299970002
63,0.200,0.0207979202079792
76,0.231,0.0007999200079992001
80,0.186,0.043695630436956304
84,0.184,0.023497650234976502
86,0.203,0.014098590140985901
91,0.168,0.0453954604539546
93,0.183,0.025097490250974904
95,0.165,0.015398460153984602
97,0.156,0.030496950304969503
105,0.158,0.04689531046895311
161,0.154,0.0216978302169783
167,0.132,0.0332966703329667
175,0.151,0.029197080291970802
180,0.166,0.036396360363963605
181,0.259,0.00039996000399960006
184,0.160,0.021397860213978603
185,0.149,0.0327967203279672
186,0.225,0.0018998100189981002
192,0.193,0.007699230076992301
193,0.153,0.044495550444955505
197,0.247,0.0007999200079992001
198,0.233,0.004199580041995801


In [19]:
rlist2, plist2 = [],[]
for roi in range(199):
    r, p = mantel_with_multiple_controls(attitude_change_SM, brain_ISC_Z_np[roi],
                                        standardize=True, corr_type='pearson',
                                        # control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                        permutations=10000,tail=2)
    if p <0.05:
        print(f'{roi},{r:.3f},{p}')

    rlist2.append(r)
    plist2.append(p)

5,0.177,0.013098690130986902
13,0.162,0.04129587041295871
22,0.178,0.0354964503549645
27,0.148,0.04939506049395061
31,0.157,0.0263973602639736
34,0.178,0.020397960203979604
51,0.145,0.0390960903909609
57,0.226,0.004399560043995601
63,0.204,0.0168983101689831
76,0.237,0.00019998000199980003
80,0.187,0.042195780421957804
84,0.188,0.024897510248975102
86,0.208,0.0122987701229877
91,0.176,0.046495350464953504
93,0.188,0.0244975502449755
95,0.169,0.012898710128987101
97,0.162,0.030396960303969604
105,0.159,0.04789521047895211
161,0.155,0.0184981501849815
167,0.138,0.027797220277972202
175,0.159,0.0216978302169783
180,0.172,0.037396260373962605
181,0.263,0.0004999500049995
184,0.162,0.023697630236976304
185,0.154,0.0263973602639736
186,0.226,0.0022997700229977
192,0.201,0.0059994000599940004
193,0.160,0.043795620437956206
197,0.252,0.0006999300069993001
198,0.237,0.005599440055994401


In [21]:
rlist3, plist3 = [],[]
for roi in range(199):
    r, p = mantel_with_multiple_controls(attitude_change_SM, brain_ISC_Z_np[roi],
                                        standardize=True, corr_type='spearman',
                                        control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                        permutations=10000,tail=2)
    if p <0.05:
        print(f'{roi},{r:.3f},{p}')

    rlist3.append(r)
    plist3.append(p)

5,0.160,0.0226977302269773
22,0.175,0.0314968503149685
31,0.151,0.024297570242975703
34,0.179,0.0170982901709829
37,0.164,0.043795620437956206
57,0.202,0.0066993300669933005
63,0.194,0.019698030196980302
76,0.224,0.0012998700129987
80,0.182,0.04689531046895311
84,0.174,0.037196280371962806
86,0.194,0.016798320167983202
93,0.167,0.043995600439956005
95,0.147,0.024997500249975
97,0.141,0.049295070492950704
161,0.155,0.0174982501749825
175,0.138,0.0402959704029597
181,0.255,0.00039996000399960006
184,0.158,0.0206979302069793
185,0.159,0.022797720227977204
186,0.225,0.0022997700229977
192,0.180,0.012698730126987301
197,0.226,0.0006999300069993001
198,0.229,0.0045995400459954


In [22]:
rlist4, plist4 = [],[]
for roi in range(199):
    r, p = mantel_with_multiple_controls(attitude_change_SM, brain_ISC_Z_np[roi],
                                        standardize=True, corr_type='spearman',
                                        # control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                        permutations=10000,tail=2)
    if p <0.05:
        print(f'{roi},{r:.3f},{p}')

    rlist4.append(r)
    plist4.append(p)

5,0.161,0.0187981201879812
22,0.182,0.029097090290970903
31,0.151,0.029497050294970503
34,0.181,0.014998500149985002
57,0.204,0.008999100089991
63,0.195,0.0198980101989801
69,0.126,0.0430956904309569
76,0.229,0.0011998800119988001
84,0.175,0.0380961903809619
86,0.193,0.0168983101689831
93,0.169,0.0438956104389561
95,0.144,0.029797020297970205
161,0.154,0.020897910208979104
175,0.138,0.0430956904309569
181,0.259,0.00039996000399960006
184,0.159,0.023397660233976603
185,0.161,0.0170982901709829
186,0.226,0.0026997300269973002
192,0.189,0.009999000099990002
197,0.226,0.0011998800119988001
198,0.227,0.0059994000599940004


In [23]:
result_data = {
    'pearson control': rlist1,
    'p1': plist1,
    'pearson no-control': rlist2,
    'p2': plist2,
    'spearman control': rlist3,
    'p3': plist3,
    'spearman no-control': rlist4,
    'p4': plist4
}

result_data_df = pd.DataFrame(result_data)

result_data_df.to_csv('/Users/li/Desktop/debate2025/results/ISFC-ISRSA-Exp1-seed89-2min.csv', index=False)